In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px

# Merging des fichiers des gares et de leurs fréquentations

On charge les fichiers de formes et des vitesses (voir ```4_frequentation_gares.ipynb``` et ```5_liste_gares.ipynb```) 

In [2]:
def process_frequentations(frequentations_df : pd.DataFrame) -> pd.DataFrame:
    """
    Voir notebooks/4_frequentation_gares.ipynb
    Traitement des données de frequentation-gares.csv
    
    La transformation consiste à passer d'un format large, où chaque année est une colonne distincte, 
    à un format long, où chaque année est une ligne distincte. Cela facilite les comparaisons et l'analyse 
    des données sur plusieurs années.
    
    Args:
        frequentations_df (pd.DataFrame): DataFrame contenant les données de frequentation-gares.csv
    Returns:
        pd.DataFrame: Dataframe traitée.
    """
    years = [str(year) for year in range(2015, 2024)]
    frequentations_df_processed = pd.DataFrame()
    for year in years:
        year_df = frequentations_df[["Nom de la gare", "Code UIC", "Code postal", "Segmentation DRG", f"Total Voyageurs {year}", f"Total Voyageurs + Non voyageurs {year}"]]
        year_df = year_df.assign(Année=year) # On peut faire year_df["Année"] = year mais c'est moins propre, on a le warning SettingWithCopyWarning.
        year_df = year_df.rename(columns={f"Total Voyageurs {year}":"Total Voyageurs", f"Total Voyageurs + Non voyageurs {year}":"Total Voyageurs + Non Voyageurs"})
        frequentations_df_processed = pd.concat([frequentations_df_processed, year_df])
    frequentations_df_processed = frequentations_df_processed.sort_values(by=["Nom de la gare", "Année"]) # On trie par nom de gare et année pour avoir un affichage plus lisible
    frequentations_df_processed = frequentations_df_processed.rename(columns={"Code UIC":"code_uic"}) # On renomme la colonne "Code UIC" en "code_uic" pour correspondre à liste-des-gares.geojson
    frequentations_df_processed["Code postal"] = frequentations_df_processed["Code postal"].astype(str) # On convertit le code postal en chaîne de caractères pour éviter les problèmes de formatage
    frequentations_df_processed = frequentations_df_processed.drop(columns=["Nom de la gare"])
    frequentations_df_processed = frequentations_df_processed.reset_index(drop=True)
    return frequentations_df_processed

frequentations = pd.read_csv('../data/raw/frequentation-gares.csv', sep=';')
frequentations_processed = process_frequentations(frequentations)
print(frequentations_processed.shape)
frequentations_processed.head()

(27090, 6)


,code_uic,Code postal,Segmentation DRG,Total Voyageurs,Total Voyageurs + Non Voyageurs,Année
0,87313759,60220,C,39720,39720,2015
1,87313759,60220,C,41096,41096,2016
2,87313759,60220,C,43760,43760,2017
3,87313759,60220,C,40228,40228,2018
4,87313759,60220,C,42685,42685,2019


In [3]:
def process_gares(gares_df : pd.DataFrame) -> pd.DataFrame:
    """
    Voir notebooks/5_liste_gares.ipynb
    Traitement des données de liste-des-gares.geojson
    On ne garde que les gares qui sont ouvertes aux voyageurs et qui sont exploitées par la SNCF.
    
    Args:
        gares_df (pd.DataFrame): DataFrame contenant les données de liste-des-gares.geojson
    Returns:
        pd.DataFrame: Dataframe traitée.
    """
    gares_processed_df = gares_df.query("voyageurs == 'O'")
    gares_processed_df = gares_processed_df.drop(columns=["voyageurs"])
    gares_processed_df = gares_processed_df.reset_index(drop=True)
    gares_processed_df["fret"] = gares_processed_df["fret"].apply(lambda x: x == "O") # Par défaut, la colonne fret est un object, on la convertit en booléen
    # fret contient des valeurs "O" et "N", on les remplace par True et False

    # On ne garde que les colonnes qui nous intéressent
    relevant_columns = ["code_uic", "libelle", "fret", "code_ligne", "geometry"]
    gares_processed_df = gares_processed_df[relevant_columns].copy() # On garde une copie pour éviter de modifier l'original
    gares_processed_df = gares_processed_df.drop_duplicates(subset=["code_uic"]) # On supprime les doublons sur le code UIC, car il y a parfois plusieurs lignes pour une même gare
    gares_processed_df["code_uic"] = gares_processed_df["code_uic"].astype("Int64") # On convertit le code UIC en entier pour éviter les problèmes de type
    return gares_processed_df

gares = gpd.read_file('../data/raw/liste-des-gares.geojson')
gares_processed = process_gares(gares)
print(gares_processed.shape)
gares_processed.head()

(2974, 5)


,code_uic,libelle,fret,code_ligne,geometry
0,87009696,La Douzillère,False,594000,POINT (0.653 47.33866)
1,87382218,La Défense,False,973000,POINT (2.23847 48.89344)
2,87718122,Byans,False,871000,POINT (5.85209 47.11833)
3,87721829,Chamelet,False,775000,POINT (4.50702 45.98167)
4,87471060,L'Hermitage-Mordelles,True,420000,POINT (-1.81921 48.12334)


## Merging des gares et de leurs fréquentations

In [4]:
def merge_gares_frequentations(gares_df : pd.DataFrame, frequentations_df : pd.DataFrame) -> gpd.GeoDataFrame:
    merged_df = gares_df.merge(frequentations_df, on="code_uic", how="left") 
    merged_df = merged_df.rename(columns={"Code postal":"code_postal"})
    merged_df["code_postal"] = merged_df["code_postal"].astype("Int64") # On convertit le code postal en entier pour éviter les problèmes de type
    merged_df = gpd.GeoDataFrame(merged_df, geometry=merged_df.geometry) # On convertit le DataFrame en GeoDataFrame
    return merged_df

gares_frequentations = merge_gares_frequentations(gares_processed, frequentations_processed)
print(gares_frequentations.shape)
gares_frequentations.head()    

(26438, 10)


,code_uic,libelle,fret,code_ligne,geometry,code_postal,Segmentation DRG,Total Voyageurs,Total Voyageurs + Non Voyageurs,Année
0,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,17749.0,17749.0,2015
1,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,13502.0,13502.0,2016
2,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,11507.0,11507.0,2017
3,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,11104.0,11104.0,2018
4,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,10886.0,10886.0,2019


## Communes de France

On aimerait également utiliser la liste des communes de france et leur population pour exploiter les différences entre la fréquentation des gares et la population des communes. Les fichiers contenant ces données sont ```20230823-communes-departement-region.csv``` et ```insee-pop-communes.csv```.
On va donc les charger et les fusionner avec le fichier des gares et des fréquentations.

In [5]:
communes = pd.read_csv('../data/raw/20230823-communes-departement-region.csv')
print(communes.shape)
communes.head()

(39201, 15)


,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


In [6]:
population = pd.read_csv('../data/raw/insee-pop-communes.csv', sep=';')
print(population.shape)
population.head()

(34995, 5)


,DEPCOM,COM,PMUN,PCAP,PTOT
0,01001,L' Abergement-Clémenciat,776,18,794
1,01002,L' Abergement-de-Varey,248,1,249
2,01004,Ambérieu-en-Bugey,14035,393,14428
3,01005,Ambérieux-en-Dombes,1689,34,1723
4,01006,Ambléon,111,6,117


La colonne ```PTOT``` désigne la population totale de la commune.

On effectue un merging des deux dataframes sur la colonne ```code_commune_INSEE```, ou ```DEPCOM```.

In [ ]:
def treat_and_merge_communes_population(communes_df : pd.DataFrame, population_df : pd.DataFrame) -> pd.DataFrame:
    communes_df = communes_df.copy()
    population_df = population_df.copy()
    
    population_df = population_df.rename(columns={"DEPCOM" : "code_commune_INSEE"})
    
    communes_df["code_commune_INSEE"] = communes_df["code_commune_INSEE"].astype(str).str.zfill(5) # On s'assure que le code commune est bien au format 5 chiffres
    
    communes_population_df = communes_df.merge(population_df, how="left", on="code_commune_INSEE")
    
    relevant_columns = ["code_commune_INSEE", "nom_commune", "code_postal", "code_departement", "nom_departement", "nom_region", "PTOT"]
    
    communes_population_df = communes_population_df[relevant_columns].copy() # On garde une copie pour éviter de modifier l'original
    
    return communes_population_df

communes_population = treat_and_merge_communes_population(communes, population)
print(communes_population.shape)
communes_population.head()

(39201, 7)


,code_commune_INSEE,nom_commune,code_postal,code_departement,nom_departement,nom_region,PTOT
0,01001,Abergement-Clémenciat,1400,1,Ain,Auvergne-Rhône-Alpes,794.0
1,01002,Abergement-de-Varey,1640,1,Ain,Auvergne-Rhône-Alpes,249.0
2,01004,Ambérieu-en-Bugey,1500,1,Ain,Auvergne-Rhône-Alpes,14428.0
3,01005,Ambérieux-en-Dombes,1330,1,Ain,Auvergne-Rhône-Alpes,1723.0
4,01006,Ambléon,1300,1,Ain,Auvergne-Rhône-Alpes,117.0


## Merging des fréquentations des gares et des communes

In [8]:
def merge_gares_communes(gares_frequentations_df: gpd.GeoDataFrame, communes_population_df: pd.DataFrame) -> gpd.GeoDataFrame:
    merged_df = gares_frequentations_df.merge(communes_population_df, on="code_postal", how="left")
    merged_df = merged_df.drop_duplicates(subset=["code_uic", "Année"])
    return merged_df

gares_communes = merge_gares_communes(gares_frequentations, communes_population)
print(gares_communes.shape)
gares_communes.head()


(26438, 16)


,code_uic,libelle,fret,code_ligne,geometry,code_postal,Segmentation DRG,Total Voyageurs,Total Voyageurs + Non Voyageurs,Année,code_commune_INSEE,nom_commune,code_departement,nom_departement,nom_region,PTOT
0,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,17749.0,17749.0,2015,37122,Joué-lès-Tours,37,Indre-et-Loire,Centre-Val de Loire,38340.0
1,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,13502.0,13502.0,2016,37122,Joué-lès-Tours,37,Indre-et-Loire,Centre-Val de Loire,38340.0
2,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,11507.0,11507.0,2017,37122,Joué-lès-Tours,37,Indre-et-Loire,Centre-Val de Loire,38340.0
3,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,11104.0,11104.0,2018,37122,Joué-lès-Tours,37,Indre-et-Loire,Centre-Val de Loire,38340.0
4,87009696,La Douzillère,False,594000,POINT (0.653 47.33866),37300,C,10886.0,10886.0,2019,37122,Joué-lès-Tours,37,Indre-et-Loire,Centre-Val de Loire,38340.0


## BONUS Simples visualisations

In [15]:
fig = px.box(
    gares_communes[(gares_communes["Année"] == "2023") & (gares_communes["nom_region"] != "Île-de-France")],
    y="Total Voyageurs",
    color="nom_region",
    points="all",
    hover_data=["libelle", "nom_commune"],
    title="Distribution du nombre de voyageurs par région (2023)"
)
fig.update_layout(yaxis_title="Total Voyageurs", xaxis_title="Région")
fig.show()

In [16]:
fig = px.scatter(
    gares_communes.query('Année == "2023" and `Total Voyageurs` > 5_000_000'),
    x="PTOT",
    y="Total Voyageurs",
    hover_name="libelle",
    color="nom_region",
    labels={
        "PTOT": "Population totale (PTOT)",
        "Total Voyageurs": "Total Voyageurs",
        "nom_region": "Région"
    },
    title="Gares à fort trafic (> 5M voyageurs) : fréquentation vs population de la commune (2023)"
)
fig.show()

In [18]:
# Filtrer les gares avec plus de 5 millions de voyageurs en 2023
df_high_trafic = gares_communes[(gares_communes["Année"] == "2023") & (gares_communes["Total Voyageurs"] > 5_000_000)]

# Compter le nombre de gares par région
region_counts = df_high_trafic["nom_region"].value_counts().reset_index()
region_counts.columns = ["Région", "Nombre de gares à fort trafic"]

# Affichage en camembert avec Plotly
fig = px.pie(
    region_counts,
    names="Région",
    values="Nombre de gares à fort trafic",
    title="Répartition des gares à forte affluence (> 5M voyageurs) par région (2023)",
    hole=0.3
)
fig.show()

In [19]:
import folium

df_non_idf = df_high_trafic[df_high_trafic["nom_region"] != "Île-de-France"]

df_non_idf["lat"] = df_non_idf.geometry.y
df_non_idf["lon"] = df_non_idf.geometry.x

# Center map on France
m = folium.Map(location=[46.6, 2.5], zoom_start=6)

max_travelers = df_non_idf["Total Voyageurs"].max()
min_travelers = df_non_idf["Total Voyageurs"].min()
def scale_size(val, min_size=5, max_size=30):
    if max_travelers == min_travelers:
        return max_size
    return min_size + (max_size - min_size) * (val - min_travelers) / (max_travelers - min_travelers)

for _, row in df_non_idf.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=scale_size(row["Total Voyageurs"]),
        color="blue",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(f"{row['libelle']}<br>{int(row['Total Voyageurs']):,} voyageurs", max_width=250)
    ).add_to(m)

m

/usr/local/Caskroom/miniconda/base/envs/python-data/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/Caskroom/miniconda/base/envs/python-data/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# Group by year and region, sum the number of travelers
region_year_travelers = gares_communes.groupby(['Année', 'nom_region'])['Total Voyageurs'].sum().reset_index()
region_year_travelers = region_year_travelers.query('nom_region != "Île-de-France"') # Exclude Île-de-France for the plot
# Convert Année to integer for correct sorting
region_year_travelers['Année'] = region_year_travelers['Année'].astype(int)

# Plot with plotly express
fig = px.line(
    region_year_travelers.sort_values(['nom_region', 'Année']),
    x='Année',
    y='Total Voyageurs',
    color='nom_region',
    markers=True,
    labels={
        'Année': 'Année',
        'Total Voyageurs': 'Total Voyageurs',
        'nom_region': 'Région'
    },
    title='Nombre total de voyageurs par région (2015-2023)'
)
fig.show()

In [21]:
region_year_travelers = gares_communes.groupby(['Année', 'nom_region'])['Total Voyageurs'].sum().reset_index()
region_year_travelers_2019_2020 = region_year_travelers.query('Année in ["2019", "2020"]').reset_index(drop=True)

region_year_travelers_loss = region_year_travelers_2019_2020.pivot(index='nom_region', columns='Année', values='Total Voyageurs').reset_index()

region_year_travelers_loss["relative_loss"] = ((region_year_travelers_loss["2019"] - region_year_travelers_loss["2020"]) / region_year_travelers_loss["2019"] * 100).round(2)

df_loss = region_year_travelers_loss.copy()
df_loss = df_loss.sort_values("relative_loss", ascending=False)

fig = px.bar(
    df_loss,
    x="nom_region",
    y="relative_loss",
    labels={"nom_region": "Région", "relative_loss": "Perte relative (%)"},
    title="Perte relative de voyageurs par région dûe au COVID (entre 2019 et 2020)",
    color="relative_loss",
    color_continuous_scale="Sunsetdark"
)
fig.update_layout(xaxis_tickangle=45)
fig.show()